In [41]:
codigo_canhero = 18358

In [42]:
from werkzeug.wrappers import Request, Response
#from flask import Flask, render_template
from jinja2 import Template

#from datetime import date
from datetime import datetime, timedelta

import collections
from amigocloud import AmigoCloud

from datetime import datetime

In [43]:
acceso = open('../api_key/api.key','r')
api_token = acceso.readlines()[0]
acceso.close()

amigocloud = AmigoCloud(token=api_token)
amigocloud

In [44]:
# DATOS DE LAS INPECCION
query = {'query': 'select * from dataset_296354 where cast(split_part(canhero, \' / \', 1) as integer) = {cod_ca}'.format(cod_ca=codigo_canhero)}
project = 'https://app.amigocloud.com/api/v1/projects/32082/sql'
select = amigocloud.get(project, query)
data = select['data'][0]

data['fecha_de_registro'] = datetime.strptime(data['fecha_de_registro'][0:10], '%Y-%m-%d').strftime('%d/%m/%Y')
#data['fecha_quema'] = datetime.strptime(data['fecha_quema'][0:10], '%Y-%m-%d').strftime('%d/%m/%Y')
#data['fecha_inspeccion'] = datetime.strptime(data['fecha_inspeccion'][0:10], '%Y-%m-%d').strftime('%d/%m/%Y')

insp = collections.namedtuple("insp", data.keys())(*data.values())
print('end')

end


In [45]:
insp

insp(fecha_de_registro='18/07/2022', amigo_id='c1b3a3bf489b4ad880864c4282a9aa87', ubicacion='0101000020E6100000F113BD42D58A4FC0769CFDDC2B4031C0', cod_prop=320, nom_prop='RIO_TORO--RIC_PARADA', cosechadora='Jd 3520', codigo_maquina='CJ03', nombre_operador='Patin', tiempo_experiencia=5, op_01=False, op_02=True, op_03=True, op_04=True, op_05=True, op_06=True, op_07=False, op_08=True, op_09=True, op_10=True, op_11=True, op_12=True, op_13=True, op_14=True, op_15=True, op_16=False, op_17=True, op_18=True, op_19=True, op_20=True, op_21=True, op_22=True, op_23=True, maq_01=True, maq_02=True, maq_03=False, maq_04=True, maq_05=True, maq_06=True, maq_07=True, maq_08=True, maq_09=True, maq_10=True, maq_11=True, maq_12=True, maq_13=True, canhero='18358 / ROJAS AMELUNGE CARLOS', nota_operador=86.9565217391304, nota_maquina=92.3076923076923, op_01_orientacion='No esta corre ta debido al terreno esta muy humedo y el operador opto por dejR  orte mas alto.', op_02_orientacion=None, op_03_orientacion=Non

In [53]:
query_fotos = {'query': 'select source_amigo_id, s3_filename from gallery_43414 where source_amigo_id=\'{source_amigo_id}\''.format(source_amigo_id=insp.amigo_id)}
project = 'https://app.amigocloud.com/api/v1/projects/32082/sql'
fotos = amigocloud.get(project, query_fotos)['data']

#if fotos_filename['data'] != []:
#    dic = {'lote': lote.lote, 'file_name': fotos_filename['data'][0]['s3_filename']}
#    fotos.append(dic)

#fotos

In [54]:
fotos

[{'source_amigo_id': 'c1b3a3bf489b4ad880864c4282a9aa87',
  's3_filename': 'WhatsApp Image 2022-07-18 at 5.49.15 PM.jpeg'},
 {'source_amigo_id': 'c1b3a3bf489b4ad880864c4282a9aa87',
  's3_filename': 'WhatsApp Image 2022-07-18 at 5.49.16 PM.jpeg'},
 {'source_amigo_id': 'c1b3a3bf489b4ad880864c4282a9aa87',
  's3_filename': 'WhatsApp Image 2022-07-18 at 5.49.16 PM (1).jpeg'}]

In [62]:
from docxtpl import DocxTemplate
import docxtpl
from docx.shared import Mm

import requests

doc = DocxTemplate("templates/01_template_operador.docx")

#descargar fotos y generar lista InlineImage

lista_fotos_inline = []
for foto in fotos:
    #response = requests.get("https://www-amigocloud.s3.amazonaws.com/gallery/" + foto['file_name'])
    #response = requests.get("https://www-amigocloud.s3.amazonaws.com/gallery/" + foto['s3_filename'])
    #file = open('fotos/' + foto['s3_filename'], "wb")
    #file.write(response.content)
    #file.close()
    lista_fotos_inline.append({'foto': docxtpl.InlineImage(doc, image_descriptor='fotos/' + foto['s3_filename'], width=Mm(50))})

context = {'insp':insp, 'fotos':lista_fotos_inline}

doc.render(context)

# formato de nombre de archivo: "123_CQ_01-01-2022_NOMBRE"
cod_nom = insp.canhero.split(' / ')
file_name = cod_nom[0] + '_EC_' + insp.fecha_de_registro.replace('/','-') + '_' + cod_nom[1]

doc.save('_' + file_name + '.docx')